In [53]:
import mediapipe as mp
import numpy as np
import cv2
import pyautogui
from time import sleep
from threading import Timer
import ctypes


def jump_function():
    global jump
    jump = True



WINDOW_NAME = 'Full Integration'


# initialize video capture object to read video from external webcam
video_capture = cv2.VideoCapture(1)
# if there is no external camera then take the built-in camera
if not video_capture.read()[0]:
    video_capture = cv2.VideoCapture(0)
    

hands = mp.solutions.hands
facmesh = mp.solutions.face_mesh
face = facmesh.FaceMesh()
hands_mesh = hands.Hands()
draw = mp.solutions.drawing_utils

jump = True
h = 0
w = 0
while True:

    user32 = ctypes.windll.user32
    screen_width, screen_height = user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)
 
 # read video frame by frame
    ret, frame = video_capture.read()

    frame = cv2.flip(frame, 1)

    frame_height, frame_width, _ = frame.shape

    scaleWidth = float(screen_width)/float(frame_width)
    scaleHeight = float(screen_height)/float(frame_height)

    if scaleHeight>scaleWidth:
        imgScale = scaleWidth

    else:
        imgScale = scaleHeight

    newX,newY = frame.shape[1]*imgScale, frame.shape[0]*imgScale

    frm = cv2.resize(frame,(int(newX),int(newY)))
    rgb = cv2.cvtColor(frm, cv2.COLOR_BGR2RGB)
      
    op2 = hands_mesh.process(rgb)
    
    if op2.multi_hand_landmarks:
        for handLms in op2.multi_hand_landmarks:
            p4y = 1000
            for id, lm in enumerate(handLms.landmark):
                if id == 4:
                    p4y = lm.y
                    
                if p4y*(int(newY)) < 600 and jump:
                    jump = False
                    t = Timer(0.3, jump_function)
                    t.start()
                    pyautogui.press('up')
            draw.draw_landmarks(frm, handLms, hands.HAND_CONNECTIONS)
           

    cv2.imshow("window", frm)

    if cv2.waitKey(1) == 27:
        video_capture.release()
        cv2.destroyAllWindows()
        break